In [1]:
import nltk
from nltk.corpus import semcor
import spacy
from nltk.corpus import wordnet

import sys
sys.path.append('..')
from utils import utils as u

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\riccardo.gino\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\riccardo.gino\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [24]:
frasi = ['ciccio trippo mangia la trippa']

for f in semcor.sents():
    frasi.append(u.remove_punctuation_pos(u.remove_stop_words_pos(u.get_lemmatized_tokens_list_pos(f))))

In [26]:
verbs = {}

for f in frasi:
    for token, tag in f:
        if tag == 'v':
            if token not in verbs:
                verbs[token] = 1
            else:
                verbs[token] += 1

verbs = sorted(verbs.items(), key=lambda x: x[1],  reverse=True)

In [27]:
def get_sents_with_verb(sents, verb):
    selected_sents = []
    for sentence in sents:
        for token, tag in sentence:
            if token == verb and tag == 'v':
                selected_sents.append(sentence)

    return selected_sents

In [31]:
# verb find
find_sents = get_sents_with_verb(frasi, 'find')


print('sono un ciccione, ho mangiato tutto il panettoneeee')

In [49]:
def get_subject(verb, sentence):
    i = 0
    subject = None

    for i in range(len(sentence)):
        if sentence[i][0] == verb:
            j = i-1
            while j >= 0:
                if sentence[j][1] == 'n':
                    subject = sentence[j]
                    return subject
                j -= 1
            j = i-1
            while j >= 0:
                if sentence[j][1] == 'unk':
                    subject = sentence[j]
                    return subject
                j -= 1
    
    return subject



In [42]:
def get_object(verb, sentence):
    i = 0
    object = None

    for i in range(len(sentence)):
        if sentence[i][0] == verb:
            j = i+1
            while j < len(sentence):
                if sentence[j][1] == 'n':
                    subject = sentence[j]
                    return subject
                j += 1
    
    return object

In [50]:
args_1 = {}
args_2 = {}

z = 1

for sentence in find_sents:
    print('Sentence:', sentence)

    args_1["slot_"+str(z)] = get_subject('find', sentence)
    print('Subject:', args_1["slot_"+str(z)])
    args_2["slot_"+str(z)] = get_object('find', sentence)
    print('Object:', args_2["slot_"+str(z)])
    z += 1


Sentence: [('The', 'unk'), ('jury', 'n'), ('say', 'v'), ('find', 'v'), ('many', 'a'), ('Georgia', 'n'), ('registration', 'n'), ('election', 'n'), ('law', 'n'), ('outmode', 'v'), ('inadequate', 'a'), ('often', 'r'), ('ambiguous', 'a')]
Subject: ('jury', 'n')
Object: ('Georgia', 'n')
Sentence: [('The', 'unk'), ('jury', 'n'), ('say', 'v'), ('find', 'v'), ('court', 'n'), ('incorporate', 'v'), ('operating', 'n'), ('procedures', 'v'), ('recommendation', 'n'), ('two', 'unk'), ('previous', 'a'), ('grand', 'a'), ('jury', 'n'), ('Atlanta', 'n'), ('Bar', 'n'), ('Association', 'n'), ('interim', 'a'), ('citizen', 'n'), ('committee', 'n')]
Subject: ('jury', 'n')
Object: ('court', 'n')
Sentence: [('Buffalo', 'n'), ('coach', 'n'), ('Buster', 'n'), ('Ramsey', 'n'), ('become', 'v'), ('one', 'unk'), ('game', 'n'), ('great', 'a'), ('collector', 'n'), ('quarterback', 'n'), ('apparently', 'r'), ('find', 'v'), ('productive', 'a'), ('pair', 'n'), ('two', 'unk'), ('ex', 'a'), ('National', 'n'), ('Football', 'n

In [9]:
# Carica il modello di lingua SpaCy
nlp = spacy.load("en_core_web_sm")

# Estrai le frasi da SemCor
sentences = semcor.sents()

# Converte le liste di token in stringhe

corpus = []
for sentence in sentences:
    
    sentence_text = ' '.join(sentence)
    doc = nlp(sentence_text)
    if any(token.lemma_ == "find" for token in doc):
        corpus.append(sentence_text)

print(len(corpus))



724


In [25]:
# Dizionario per raccogliere i tipi semantici dei ruoli
semantic_roles = {"nsubj": {}, "dobj": {}}

# Analisi delle frasi nel corpus
for sentence in corpus:
    doc = nlp(sentence)

    i = 1
    
    # Estrai ruoli sintattici e tipi semantici
    for token in doc:
        if token.dep_ == "nsubj" or token.dep_ == "dobj":
           
            word = u.lemmatize_word(token.text, 'n')
            
            # Mappatura a tipi semantici utilizzando WordNet (solo come esempio)
            semantic_roles[token.dep_]['slot_'+str(i)]  = wordnet.synsets(word, 'n')
            i+=1


In [26]:
for slot in semantic_roles['nsubj']:
    print(slot)

slot_1
slot_2
slot_3
slot_4
slot_6
slot_7
slot_9
slot_5
slot_10
slot_8
slot_11
slot_12
slot_13
